In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/gdrive/")
    %cd "/content/gdrive/My Drive/Colab Notebooks/bikes-theft-model"

# Libraries: Standard ones
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

# Library for boxplots
import seaborn as sns
import pandas as pd

#GRAPHS CLASS
from Codigos.DataStatistics import GraphsStatistics as gp

In [ ]:
data = pd.read_csv('Data/Bicycle_Theft_Ottawa.csv', index_col= 0)
display(data.sample(6))


In [ ]:
data.shape

In [ ]:
data.describe()
gp.CountBar_plot

In [ ]:
data.describe(include='O')

In [ ]:
data.info()

In [ ]:
data.convert_dtypes().dtypes

In [ ]:
data[['Neighbourhood', 'Bicycle_Status']].head(10) #if 'Bicycle_Status' == 'STOLEN'

In [ ]:
data[data['Bicycle_Status'] == 'STOLEN'].groupby("Neighbourhood").size().idxmax()

In [ ]:
data.columns

In [ ]:
 #Y = data.loc[data['Bicycle_Status'] == 'STOLEN'].groupby("Incident_Weekday").size()
 # data.groupby("Neighbourhood").size()
sub_Da = data[data['Bicycle_Status'] == 'STOLEN'].groupby("Incident_Weekday").size().reset_index().replace({1: 'Monday',2: 'Tuesday', 3: 'Weekday', 4: 'Thursday', 5: 'Friday', 6:'Saturday', 7:'Sunday'})
sub_Da.rename(columns={0:'Stolen Bicycles'}, inplace=True)

fig = px.bar(sub_Da, "Incident_Weekday", 'Stolen Bicycles', title='Stolen Bicycles')
fig.show()

In [ ]:
data['Report_Date']=pd.to_datetime(data['Report_Date']) #Transforma la columna Formato de fecha
data['Week_day']=data['Report_Date'].dt.day_name() #Agregar el nombre
data['Year'] = data['Report_Date'].dt.year

In [ ]:
Dias_aux_robo_year=data.groupby(['Year','Bicycle_Status']).size().reset_index().rename(columns={0:'Count'})
display(Dias_aux_robo_year)

fig = px.line(Dias_aux_robo_year, x='Year', y='Count', color='Bicycle_Status')
fig.show()

# Geojson incorporation

Now, let's trying to plot a geojson file and function to join data with map:

In [26]:
import pandas as pd

import branca, geopandas
import folium # package for making maps, please make sure to use a version older than 1.0.0.
from folium.plugins import TimeSliderChoropleth

# Function
def mapJoinTable(df_point, df_map):
    try:
        df_point = geopandas.GeoDataFrame(df_point, geometry = geopandas.points_from_xy(df_point.x, df_point.y)).set_crs("EPSG:4326")
    except:
        df_point = geopandas.GeoDataFrame(df_point, geometry = geopandas.points_from_xy(df_point.Long, df_point.Lat)).set_crs("EPSG:4326")
    df_result = geopandas.tools.sjoin(df_point, df_map, how='left')
    return df_result

In [27]:
# Map creation colors and read geojson file
colormap = branca.colormap.LinearColormap(
    colors=['white','yellow','orange','red','darkred'],
    #index=beat_cn['count'].quantile([0.2,0.4,0.6,0.8]),b
#     vmin=min_cn,
#     vmax=max_cn
)
colormap.caption = "Unname"

# Read geojson/data and put in pandas dataframe
canada_data = pd.read_csv("./Data/Bicycle_Thefts_Toronto.csv")
canada_map = geopandas.read_file("./Data/Maps/toro_population_toronto.geojson", driver = "GeoJSON")
canada_map_data = mapJoinTable(canada_data, canada_map)

# For Tatiana et Diego xD
display(canada_data.head().append(canada_data.tail()), canada_data.shape)
canada_data = canada_map_data.drop(columns = ["geometry"])
canada_data.to_csv("./Data/Bicycle_Thefts_Toronto_geo.csv", index=False)

display(canada_data.head().append(canada_data.tail()), canada_data.shape)
display(canada_map.head().append(canada_map.tail()), canada_map.shape)
# display(canada_map_data.head().append(canada_map_data.tail()), canada_map_data.shape)
display(canada_data.columns)
# display(canada_map_data["geometry"])

,X,Y,FID,Index_,event_unique_id,Primary_Offence,Occurrence_Date,Occurrence_Year,Occurrence_Month,Occurrence_Day,...,Bike_Type,Bike_Speed,Bike_Colour,Cost_of_Bike,Status,Hood_ID,Neighbourhood,Lat,Long,geometry
0,-8.858872e+06,5.425243e+06,1,201,GO-20142058713,THEFT UNDER,2014/05/10 00:00:00,2014,5,10,...,MT,10,ONG,NaN,STOLEN,2,Mount Olive-Silverstone-Jamestown (2),43.739670,-79.580597,POINT (-79.58060 43.73967)
1,-8.834825e+06,5.419675e+06,2,202,GO-20142059340,PROPERTY - FOUND,2014/05/11 00:00:00,2014,5,11,...,MT,1,BLK,NaN,RECOVERED,56,Leaside-Bennington (56),43.703518,-79.364586,POINT (-79.36459 43.70352)
2,-8.836985e+06,5.413433e+06,3,203,GO-20142060810,THEFT UNDER,2014/05/11 00:00:00,2014,5,11,...,MT,18,BLU,200.00,STOLEN,76,Bay Street Corridor (76),43.662968,-79.383987,POINT (-79.38399 43.66297)
3,-8.826689e+06,5.417104e+06,4,204,GO-20142062543,THEFT UNDER,2014/05/11 00:00:00,2014,5,11,...,OT,24,WHI,1600.00,STOLEN,62,East End-Danforth (62),43.686817,-79.291496,POINT (-79.29150 43.68682)
4,-8.826689e+06,5.417104e+06,5,205,GO-20142062543,THEFT UNDER,2014/05/11 00:00:00,2014,5,11,...,OT,18,BLU,650.00,STOLEN,62,East End-Danforth (62),43.686817,-79.291496,POINT (-79.29150 43.68682)
21579,-8.833619e+06,5.412199e+06,21580,21376,GO-20199033967,THEFT UNDER - BICYCLE,2019/10/11 00:00:00,2019,10,11,...,MT,24,BLK,408.24,STOLEN,73,Moss Park (73),43.654949,-79.353752,POINT (-79.35375 43.65495)
21580,-8.835028e+06,5.410539e+06,21581,21377,GO-20199033985,THEFT UNDER - BICYCLE,2019/10/10 00:00:00,2019,10,10,...,RG,12,LBL,500.00,STOLEN,77,Waterfront Communities-The Island (77),43.644157,-79.366409,POINT (-79.36641 43.64416)
21581,-8.841614e+06,5.408926e+06,21582,21378,GO-20199034019,THEFT UNDER - BICYCLE,2019/10/12 00:00:00,2019,10,12,...,RC,22,ONG,3000.00,STOLEN,82,Niagara (82),43.633675,-79.425568,POINT (-79.42557 43.63367)
21582,-8.836874e+06,5.410296e+06,21583,21379,GO-20199034036,THEFT UNDER,2019/09/01 00:00:00,2019,9,1,...,RG,18,BLK,200.00,STOLEN,77,Waterfront Communities-The Island (77),43.642578,-79.382988,POINT (-79.38299 43.64258)
21583,-8.854264e+06,5.416021e+06,21584,21380,GO-20199034037,THEFT UNDER - BICYCLE,2019/10/15 00:00:00,2019,10,15,...,MT,1,BLK,750.00,STOLEN,10,Princess-Rosethorn (10),43.679783,-79.539207,POINT (-79.53921 43.67978)


(21584, 27)

,X,Y,FID,Index_,event_unique_id,Primary_Offence,Occurrence_Date,Occurrence_Year,Occurrence_Month,Occurrence_Day,...,t,dw,hh,id,pop,pop2,rgid,rpid,ruid,rguid
0,-8.858872e+06,5.425243e+06,1,201,GO-20142058713,THEFT UNDER,2014/05/10 00:00:00,2014,5,10,...,CT,1864,1784,5350248.02,5671,5724,35,35535,3520005,3520
1,-8.834825e+06,5.419675e+06,2,202,GO-20142059340,PROPERTY - FOUND,2014/05/11 00:00:00,2014,5,11,...,CT,2376,2250,5350196.02,5635,5748,35,35535,3520005,3520
2,-8.836985e+06,5.413433e+06,3,203,GO-20142060810,THEFT UNDER,2014/05/11 00:00:00,2014,5,11,...,CT,6837,5374,5350062.02,9084,6461,35,35535,3520005,3520
3,-8.826689e+06,5.417104e+06,4,204,GO-20142062543,THEFT UNDER,2014/05/11 00:00:00,2014,5,11,...,CT,2986,2887,5350079.00,7327,6991,35,35535,3520005,3520
4,-8.826689e+06,5.417104e+06,5,205,GO-20142062543,THEFT UNDER,2014/05/11 00:00:00,2014,5,11,...,CT,2986,2887,5350079.00,7327,6991,35,35535,3520005,3520
21579,-8.833619e+06,5.412199e+06,21580,21376,GO-20199033967,THEFT UNDER - BICYCLE,2019/10/11 00:00:00,2019,10,11,...,CT,5250,4663,5350017.00,7906,6883,35,35535,3520005,3520
21580,-8.835028e+06,5.410539e+06,21581,21377,GO-20199033985,THEFT UNDER - BICYCLE,2019/10/10 00:00:00,2019,10,10,...,CT,5250,4663,5350017.00,7906,6883,35,35535,3520005,3520
21581,-8.841614e+06,5.408926e+06,21582,21378,GO-20199034019,THEFT UNDER - BICYCLE,2019/10/12 00:00:00,2019,10,12,...,CT,6390,6060,5350008.02,9942,6553,35,35535,3520005,3520
21582,-8.836874e+06,5.410296e+06,21583,21379,GO-20199034036,THEFT UNDER,2019/09/01 00:00:00,2019,9,1,...,CT,5997,5311,5350012.03,8865,5892,35,35535,3520005,3520
21583,-8.854264e+06,5.416021e+06,21584,21380,GO-20199034037,THEFT UNDER - BICYCLE,2019/10/15 00:00:00,2019,10,15,...,CT,1348,1306,5350232.00,3825,3995,35,35535,3520005,3520


(21584, 38)

,a,t,dw,hh,id,pop,pop2,rgid,rpid,ruid,rguid,geometry
0,0.73445,CT,1246,1143,5350160.00,3116,3230,35,35535,3520005,3520,"MULTIPOLYGON (((-79.46018 43.68152, -79.46042 ..."
1,0.97486,CT,2130,2051,5350351.01,5497,5284,35,35535,3520005,3520,"MULTIPOLYGON (((-79.27787 43.73005, -79.27804 ..."
2,0.11566,CT,2688,2535,5350102.02,4313,4301,35,35535,3520005,3520,"MULTIPOLYGON (((-79.46689 43.65319, -79.46846 ..."
3,0.20716,CT,728,684,5350028.01,1387,988,35,35535,3520005,3520,"MULTIPOLYGON (((-79.34576 43.65980, -79.34558 ..."
4,0.62823,CT,2629,2515,5350164.00,6258,6331,35,35535,3520005,3520,"MULTIPOLYGON (((-79.43183 43.68881, -79.43125 ..."
567,1.12407,CT,2117,1953,5350270.01,4980,5152,35,35535,3520005,3520,"MULTIPOLYGON (((-79.31523 43.75786, -79.31401 ..."
568,1.10548,CT,2376,2250,5350196.02,5635,5748,35,35535,3520005,3520,"MULTIPOLYGON (((-79.36929 43.71167, -79.36892 ..."
569,0.11899,CT,359,340,5350102.05,861,812,35,35535,3520005,3520,"MULTIPOLYGON (((-79.46148 43.65830, -79.46178 ..."
570,0.29723,CT,5192,4954,5350008.01,7503,2729,35,35535,3520005,3520,"MULTIPOLYGON (((-79.41949 43.63579, -79.41974 ..."
571,0.19221,CT,3761,3519,5350063.06,5171,4737,35,35535,3520005,3520,"MULTIPOLYGON (((-79.38309 43.66137, -79.38504 ..."


(572, 12)

Index(['X', 'Y', 'FID', 'Index_', 'event_unique_id', 'Primary_Offence',
       'Occurrence_Date', 'Occurrence_Year', 'Occurrence_Month',
       'Occurrence_Day', 'Occurrence_Time', 'Division', 'City',
       'Location_Type', 'Premise_Type', 'Bike_Make', 'Bike_Model', 'Bike_Type',
       'Bike_Speed', 'Bike_Colour', 'Cost_of_Bike', 'Status', 'Hood_ID',
       'Neighbourhood', 'Lat', 'Long', 'index_right', 'a', 't', 'dw', 'hh',
       'id', 'pop', 'pop2', 'rgid', 'rpid', 'ruid', 'rguid'],
      dtype='object')

In [16]:
# Map run 
m_canada = folium.Map(location=[45.289547, -75.76206], zoom_start=9, tiles="OpenStreetMap")

stategeo = folium.GeoJson(canada_map.to_json(), name='Ottawa maps',
#     tooltip=folium.GeoJsonTooltip(
#         fields=['beat_num', 'crime_count'],
#         aliases=['Beat', 'Total crime'], 
#         localize=True
#     )
).add_to(m_canada)

# stategeo = folium.GeoJson(canada_map_data.to_json(), name='Ottawa maps points',
# #     tooltip=folium.GeoJsonTooltip(
# #         fields=['beat_num', 'crime_count'],
# #         aliases=['Beat', 'Total crime'], 
# #         localize=True
# #     )
# ).add_to(m_canada)

colormap.add_to(m_canada)
# canada_map_data.plot(color='red')
m_canada